### import some libraries

In [132]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
from sklearn import linear_model
#from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import OneHotEncoder, scale, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import math
import pygeohash as pgh

### Read in Training and Testing data

In [84]:
Train_raw = pd.read_csv(r'D:\Learn\HomeWork_Lyft\train.csv')
Test_raw = pd.read_csv(r'D:\Learn\HomeWork_Lyft\test.csv')

In [117]:
Train_raw.head()

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,date_id_tmp,date_id_left
0,0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,2015-01-11,2015-01-11
1,1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,2015-01-11,2015-01-11
2,2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,2015-01-16,2015-01-16
3,3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,2015-01-16,2015-01-16
4,4,-73.945511,40.773724,-73.987434,40.755707,1422173586,705,2015-01-25,2015-01-25


In [118]:
Test_raw.head()

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,date_id_tmp,date_id_left
0,0,-73.993111,40.724289,-74.000977,40.735222,1422173589,2015-01-25,2015-01-25
1,1,-73.971924,40.762749,-73.965698,40.771427,1420567340,2015-01-06,2015-01-06
2,2,-73.953247,40.765816,-73.952843,40.772453,1420567343,2015-01-06,2015-01-06
3,3,-73.986618,40.739353,-73.949158,40.805161,1420103336,2015-01-01,2015-01-01
4,4,-73.968864,40.757317,-73.982521,40.771305,1420690180,2015-01-08,2015-01-08


### Add weather data in NYC

Source of Daily Weather data in Central Park Area:
Start date: 2015-1-1
End date: 2016-1-1 

Downloaded from:
http://w2.weather.gov/climate/xmacis.php?wfo=okx

Daily data for a whole month
- daily maximum, minimum and average temperature (degrees F), 
- average temperature departure from normal (degrees F), 
- heating and cooling degree days (base 65)
- "Heating degree days", or "HDD", are a measure of how much (in degrees), and for how long (in days), outside air temperature was lower than a specific "base temperature" (or "balance point").
- "Cooling degree days", or "CDD", are a measure of how much (in degrees), and for how long (in days), outside air temperature was higher than a specific base temperature. 
- precipitation
- snowfall and snow depth (inches) for all days of the selected month. 

In [119]:
Weather_raw = pd.read_csv(r'D:\Learn\HomeWork_Lyft\weather_data.csv')

In [120]:
Weather_raw.head()

,Date,Max_Temp,Min_Temp,Avg_Temp,Temp_Departure,HDD,CDD,Precipition,New_Snow,Snow_Depth
0,1/1/2015,39,27,33.0,-0.4,32,0,0.00,0.0,0
1,1/2/2015,42,35,38.5,5.2,26,0,0.00,0.0,0
2,1/3/2015,42,33,37.5,4.4,27,0,0.71,0.0,0
3,1/4/2015,56,41,48.5,15.5,16,0,0.30,0.0,0
4,1/5/2015,49,21,35.0,2.1,30,0,0.00,0.0,0


### Convert datetime to appropriate format to merge with weather data

In [104]:
def convert_time_input(input_df):
    input_df['date_id_tmp'] = input_df['start_timestamp'].astype('int').astype("datetime64[s]").dt.date
    dt_lst = input_df['date_id_tmp'].tolist()
    dt_lst2 = [ x.strftime('%Y-%m-%d') for x in dt_lst]
    input_df['date_id_left'] = dt_lst2
    return input_df

In [108]:
def convert_time_weather(weather_df):
    date_rng = pd.date_range('1/1/2015', periods=366, freq='D')
    dt_lst = date_rng.tolist()
    dt_lst2 = [ x.strftime('%Y-%m-%d') for x in dt_lst]
    weather_df['date_id_right'] = dt_lst2
    return weather_df

### Left outer join weather data to the training data

In [105]:
Train_raw = convert_time_input(Train_raw)
Weather_raw = convert_time_weather(Weather_raw)
Train_with_weather = pd.merge(left=Train_raw,right=Weather_raw, how='left',  left_on='date_id_left', right_on='date_id_right')

In [121]:
Train_with_weather.head()

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,date_id_tmp,date_id_left,Date,Max_Temp,Min_Temp,Avg_Temp,Temp_Departure,HDD,CDD,Precipition,New_Snow,Snow_Depth,date_id_right
0,0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,2015-01-11,2015-01-11,1/11/2015,37,18,27.5,-4.9,37,0,0.0,0.0,1,2015-01-11
1,1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,2015-01-11,2015-01-11,1/11/2015,37,18,27.5,-4.9,37,0,0.0,0.0,1,2015-01-11
2,2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,2015-01-16,2015-01-16,1/16/2015,43,20,31.5,-0.7,33,0,0.0,0.0,0,2015-01-16
3,3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,2015-01-16,2015-01-16,1/16/2015,43,20,31.5,-0.7,33,0,0.0,0.0,0,2015-01-16
4,4,-73.945511,40.773724,-73.987434,40.755707,1422173586,705,2015-01-25,2015-01-25,1/25/2015,42,31,36.5,4.0,28,0,0.0,0.0,2,2015-01-25


### To solve the high cardinality issue of longitude and latitude features, for each unique coordinates, calculate the number of trips and average duration in that location

- dict1: Lookup table for number of trips for each unique longitude/latitude combination in starting point
- dict2: Lookup table for number of trips for each unique longitude/latitude combination in ending point
- dict3: Lookup table for average duration for each unique longitude/latitude combination in starting point
- dict4: Lookup table for average duration for each unique longitude/latitude combination in ending point

**dict1 and dict2 measures the frequency of trips in each location**

**dict3 and dict4 measures the duration of trips in each location**

In [122]:
# To test running this function: 
# tr1, dct1, dct2, dct3, dct4 = create_features_training(Train_raw.loc[:10,:])

def create_features_training(input_df):
    input_df['start_lat_2'] = round(input_df['start_lat'],3)
    input_df['start_lng_2'] = round(input_df['start_lng'],3)
    input_df['end_lng_2'] = round(input_df['end_lng'],3)
    input_df['end_lat_2'] = round(input_df['end_lat'],3)
    
    trip_cnt_by_ln_lat_start = input_df.groupby(['start_lat_2','start_lng_2'])['row_id'].size()
    trip_cnt_by_ln_lat_end   = input_df.groupby(['end_lat_2','end_lng_2'])['row_id'].size()
    
    trip_cnt_by_ln_lat_start_dict = dict(trip_cnt_by_ln_lat_start.sort_values(ascending=False))
    trip_cnt_by_ln_lat_end_dict   = dict(trip_cnt_by_ln_lat_end.sort_values(ascending=False))

    trip_duration_by_ln_lat_start = input_df.groupby(['start_lat_2','start_lng_2'])['duration'].mean()
    trip_duration_by_ln_lat_end   = input_df.groupby(['end_lat_2','end_lng_2'])['duration'].mean()
    
    trip_duration_by_ln_lat_start_dict = dict(trip_duration_by_ln_lat_start.sort_values(ascending=False))
    trip_duration_by_ln_lat_end_dict   = dict(trip_duration_by_ln_lat_end.sort_values(ascending=False))
    
    lst_start_lat_lng = input_df[['start_lat_2','start_lng_2']].values.tolist()
    lst_end_lat_lng = input_df[['end_lat_2','end_lng_2']].values.tolist()
    
    lst_trip_cnt_by_ln_lat_start = [trip_cnt_by_ln_lat_start_dict.get(tuple(x)) for x in lst_start_lat_lng]
    lst_trip_cnt_by_ln_lat_end = [trip_cnt_by_ln_lat_end_dict.get(tuple(x)) for x in lst_end_lat_lng]

    lst_trip_duration_by_ln_lat_start = [trip_duration_by_ln_lat_start_dict.get(tuple(x)) for x in lst_start_lat_lng]
    lst_trip_duration_by_ln_lat_end = [trip_duration_by_ln_lat_end_dict.get(tuple(x)) for x in lst_end_lat_lng]
    
    input_df['trip_cnt_by_start'] = lst_trip_cnt_by_ln_lat_start
    input_df['trip_cnt_by_end'] = lst_trip_cnt_by_ln_lat_end
    input_df['trip_duration_by_start'] = lst_trip_duration_by_ln_lat_start
    input_df['trip_duration_by_end'] = lst_trip_duration_by_ln_lat_end
    
    return input_df, trip_cnt_by_ln_lat_start_dict, trip_cnt_by_ln_lat_end_dict, trip_duration_by_ln_lat_start_dict, trip_duration_by_ln_lat_end_dict

In [123]:
# To test running this function
# toscore = create_features_scoring(Test_raw.loc[:10,:], dct1, dct2, dct3, dct4)

def create_features_scoring(input_df, dct1, dct2, dct3, dct4):
    input_df['start_lat_2'] = round(input_df['start_lat'],3)
    input_df['start_lng_2'] = round(input_df['start_lng'],3)
    input_df['end_lng_2'] = round(input_df['end_lng'],3)
    input_df['end_lat_2'] = round(input_df['end_lat'],3)
       
    lst_start_lat_lng = input_df[['start_lat_2','start_lng_2']].values.tolist()
    lst_end_lat_lng = input_df[['end_lat_2','end_lng_2']].values.tolist()
    
    lst_trip_cnt_by_ln_lat_start = [dct1.get(tuple(x)) for x in lst_start_lat_lng]
    lst_trip_cnt_by_ln_lat_end = [dct2.get(tuple(x)) for x in lst_end_lat_lng]

    lst_trip_duration_by_ln_lat_start = [dct3.get(tuple(x)) for x in lst_start_lat_lng]
    lst_trip_duration_by_ln_lat_end = [dct4.get(tuple(x)) for x in lst_end_lat_lng]
    
    input_df['trip_cnt_by_start'] = lst_trip_cnt_by_ln_lat_start
    input_df['trip_cnt_by_end'] = lst_trip_cnt_by_ln_lat_end
    input_df['trip_duration_by_start'] = lst_trip_duration_by_ln_lat_start
    input_df['trip_duration_by_end'] = lst_trip_duration_by_ln_lat_end
    
    return input_df

In [124]:
Train_with_weather_Trip, dict1, dict2, dict3, dict4 = create_features_training(Train_with_weather)

In [125]:
Train_with_weather_Trip.head()

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,date_id_tmp,date_id_left,Date,...,Snow_Depth,date_id_right,start_lat_2,start_lng_2,end_lng_2,end_lat_2,trip_cnt_by_start,trip_cnt_by_end,trip_duration_by_start,trip_duration_by_end
0,0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,2015-01-11,2015-01-11,1/11/2015,...,1,2015-01-11,40.714,-74.009,-74.004,40.720,7987,1495,850.331414,715.309699
1,1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,2015-01-11,2015-01-11,1/11/2015,...,1,2015-01-11,40.762,-73.971,-74.004,40.743,8664,10631,764.462027,688.180322
2,2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,2015-01-16,2015-01-16,1/16/2015,...,0,2015-01-16,40.745,-73.995,-74.000,40.735,15468,4782,652.257564,675.881430
3,3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,2015-01-16,2015-01-16,1/16/2015,...,0,2015-01-16,40.750,-73.991,-73.989,40.735,64132,2986,850.666625,738.551909
4,4,-73.945511,40.773724,-73.987434,40.755707,1422173586,705,2015-01-25,2015-01-25,1/25/2015,...,2,2015-01-25,40.774,-73.946,-73.987,40.756,3584,8497,747.302734,843.338590


Now we have 4 extra continuous features that represent longitude and latitude trip info!

### Create features based on the trip's date: day of week, hour of day, IsHoliday, IsWeekend, etc.

DayOfWeek: Monday = 0, Sunday = 6

In [127]:
def create_date_features(input_df):
    input_df['start_datetime'] = input_df['start_timestamp'].astype('int').astype("datetime64[s]")
    
    input_df['start_date'] = input_df['start_datetime'].dt.date
    input_df['start_month'] = input_df['start_datetime'].dt.month
    input_df['start_dayOfWeek'] = input_df['start_datetime'].dt.dayofweek
    input_df['start_hour'] = input_df['start_datetime'].dt.hour

    input_df.loc[input_df['start_dayOfWeek'].isin([5, 6]), 'start_isWeekend'] = 1
    input_df.loc[input_df['start_dayOfWeek'].isin([0, 1, 2, 3, 4]), 'start_isWeekend'] = 0

    cal = calendar()
    holidays = cal.holidays(start=input_df['start_date'].min(), end=input_df['start_date'].max())
    input_df['start_isHoliday'] = np.where(input_df.start_datetime.dt.normalize().isin(holidays), 1, 0)
    return input_df

In [128]:
Train_with_weather_Trip_date = create_date_features(Train_with_weather_Trip)

In [131]:
Train_with_weather_Trip_date.head()

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,date_id_tmp,date_id_left,Date,...,trip_cnt_by_end,trip_duration_by_start,trip_duration_by_end,start_datetime,start_date,start_month,start_dayOfWeek,start_hour,start_isWeekend,start_isHoliday
0,0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,2015-01-11,2015-01-11,1/11/2015,...,1495,850.331414,715.309699,2015-01-11 04:33:39,2015-01-11,1,6,4,1.0,0
1,1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,2015-01-11,2015-01-11,1/11/2015,...,10631,764.462027,688.180322,2015-01-11 04:33:39,2015-01-11,1,6,4,1.0,0
2,2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,2015-01-16,2015-01-16,1/16/2015,...,4782,652.257564,675.881430,2015-01-16 03:05:41,2015-01-16,1,4,3,0.0,0
3,3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,2015-01-16,2015-01-16,1/16/2015,...,2986,850.666625,738.551909,2015-01-16 03:05:42,2015-01-16,1,4,3,0.0,0
4,4,-73.945511,40.773724,-73.987434,40.755707,1422173586,705,2015-01-25,2015-01-25,1/25/2015,...,8497,747.302734,843.338590,2015-01-25 08:13:06,2015-01-25,1,6,8,1.0,0


### Create Distance: Haversine Distance, GeoHash Distance, 

In [133]:
def haversine_dist(lat1, lon1, lat2, lon2):
    #radius = 6371 # km
    radius = 3959 #miles

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [134]:
def create_hav_dist_feature(input_df):
    lat1 = input_df['start_lat'].tolist()
    lon1 = input_df['start_lng'].tolist()
    lat2 = input_df['end_lat'].tolist()
    lon2 = input_df['end_lng'].tolist()

    tripDistance = list(map(haversine_dist, lat1, lon1, lat2, lon2))

    tripDistance_DF = pd.DataFrame({'haversineDist': tripDistance})

    input_df = pd.concat([input_df, tripDistance_DF], axis=1)
    return input_df

In [135]:
Train_with_weather_Trip_date_hav = create_hav_dist_feature(Train_with_weather_Trip_date)

In [137]:
Train_with_weather_Trip_date_hav.tail()

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,date_id_tmp,date_id_left,Date,...,trip_duration_by_start,trip_duration_by_end,start_datetime,start_date,start_month,start_dayOfWeek,start_hour,start_isWeekend,start_isHoliday,haversineDist
12905710,12905710,-73.994621,40.752434,-73.994621,40.752434,1451634880,98,2016-01-01,2016-01-01,1/1/2016,...,816.422452,989.006425,2016-01-01 07:54:40,2016-01-01,1,4,7,0.0,1,0.000000
12905711,12905711,-73.988258,40.731770,-73.983627,40.725861,1451634910,227,2016-01-01,2016-01-01,1/1/2016,...,714.532497,705.998754,2016-01-01 07:55:10,2016-01-01,1,4,7,0.0,1,0.474878
12905712,12905712,-73.994209,40.741879,-73.996269,40.742676,1451634968,43,2016-01-01,2016-01-01,1/1/2016,...,730.159291,671.619076,2016-01-01 07:56:08,2016-01-01,1,4,7,0.0,1,0.121092
12905713,12905713,-73.981796,40.778156,-73.980896,40.781826,1451634971,171,2016-01-01,2016-01-01,1/1/2016,...,669.063375,688.877747,2016-01-01 07:56:11,2016-01-01,1,4,7,0.0,1,0.257923
12905714,12905714,-73.966759,40.773186,-73.972054,40.765629,1451635024,89,2016-01-01,2016-01-01,1/1/2016,...,816.950504,760.118718,2016-01-01 07:57:04,2016-01-01,1,4,7,0.0,1,0.591136


In [142]:
def create_geohash_dist_feature(input_df):
    lat1 = input_df['start_lat'].tolist()
    lon1 = input_df['start_lng'].tolist()
    lat2 = input_df['end_lat'].tolist()
    lon2 = input_df['end_lng'].tolist()
    
    start_geohash = [pgh.encode(x,y) for x,y in zip(lat1, lon1)]
    end_geohash = [pgh.encode(x,y) for x,y in zip(lat2, lon2)]
    
    geodistance = [pgh.geohash_approximate_distance(x,y) for x,y in zip(start_geohash, end_geohash)]

    start_geohash_DF = pd.DataFrame({'start_geohash': start_geohash})
    end_geohash_DF = pd.DataFrame({'end_geohash': end_geohash})
    geodistance_DF = pd.DataFrame({'geoDist': geodistance})

    input_df = pd.concat([input_df, geodistance_DF], axis=1)
    input_df = pd.concat([input_df, start_geohash_DF], axis=1)
    input_df = pd.concat([input_df, end_geohash_DF], axis=1)
    return input_df

In [143]:
Train_with_weather_Trip_date_hav_geo = create_geohash_dist_feature(Train_with_weather_Trip_date_hav)

In [146]:
Train_with_weather_Trip_date_hav_geo.head()

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,date_id_tmp,date_id_left,Date,...,start_date,start_month,start_dayOfWeek,start_hour,start_isWeekend,start_isHoliday,haversineDist,geoDist,start_geohash,end_geohash
0,0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,2015-01-11,2015-01-11,1/11/2015,...,2015-01-11,1,6,4,1.0,0,0.493774,3803.0,dr5regsppmbb,dr5reuzen5x4
1,1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,2015-01-11,2015-01-11,1/11/2015,...,2015-01-11,1,6,4,1.0,0,2.202658,19545.0,dr5rusxf6em6,dr5rgbzz1sgp
2,2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,2015-01-16,2015-01-16,1/16/2015,...,2015-01-16,1,4,3,0.0,0,0.766369,19545.0,dr5ru1qt291p,dr5rspdbs3nh
3,3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,2015-01-16,2015-01-16,1/16/2015,...,2015-01-16,1,4,3,0.0,0,1.057838,19545.0,dr5ru6363v2z,dr5rsre1umy7
4,4,-73.945511,40.773724,-73.987434,40.755707,1422173586,705,2015-01-25,2015-01-25,1/25/2015,...,2015-01-25,1,6,8,1.0,0,2.522606,19545.0,dr5rvqds0tn9,dr5ru77gpy6w


### Distance from starting point to Downtown, direction relative to downtown (away of near)

In [159]:
def create_downtown_dist_feature(input_df):
    lng_dwt_ls = [input_df['end_lng'].median()] * input_df.shape[0]
    lat_dwt_ls = [input_df['end_lat'].median()] * input_df.shape[0]
    downtown_df = pd.DataFrame({'downtown_lng': lng_dwt_ls, 'downtown_lat': lat_dwt_ls})
    input_df = input_df.join(downtown_df)

    #distance to downtown from start
    Startlat = input_df['start_lat'].tolist()
    Startlon = input_df['start_lng'].tolist()
    lat = input_df['downtown_lat'].tolist()
    lon = input_df['downtown_lng'].tolist()

    DistanceToDowntown = list(map(haversine_dist, Startlat, Startlon, lat, lon))
    DistanceToDowntown_DF = pd.DataFrame({'DistanceToDowntown': DistanceToDowntown})
    input_df = pd.concat([input_df, DistanceToDowntown_DF], axis=1)

    return input_df

In [160]:
def calculate_direction(lat1, lng1, lat2, lng2):
    # calculate the angle between two points
    lat1, lng1 = math.radians(lat1), math.radians(lng1)
    lat2, lng2 = math.radians(lat2), math.radians(lng2)
    aa = np.sin(lng2 - lng1) * np.cos(lat2)
    bb = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng2 - lng1)
    return np.arctan2(aa, bb) + np.pi 

In [161]:
def create_downtown_direction_feature(input_df):
    #distance to downtown from start
    Startlat = input_df['start_lat'].tolist()
    Startlon = input_df['start_lng'].tolist()
    lat = input_df['downtown_lat'].tolist()
    lon = input_df['downtown_lng'].tolist()

    DirectionToDowntown = list(map(calculate_direction, Startlat, Startlon, lat, lon))

    DirectionToDowntown_DF = pd.DataFrame({'DirectionToDowntown': DirectionToDowntown})

    input_df = pd.concat([input_df, DirectionToDowntown_DF], axis=1)
    return input_df


In [162]:
Train_with_weather_Trip_date_hav_geo_dtw = create_downtown_dist_feature(Train_with_weather_Trip_date_hav_geo)


In [163]:
Train_with_weather_Trip_date_hav_geo_dtw.head()

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,date_id_tmp,date_id_left,Date,...,start_hour,start_isWeekend,start_isHoliday,haversineDist,geoDist,start_geohash,end_geohash,downtown_lat,downtown_lng,DistanceToDowntown
0,0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,2015-01-11,2015-01-11,1/11/2015,...,4,1.0,0,0.493774,3803.0,dr5regsppmbb,dr5reuzen5x4,40.754078,-73.980087,3.169278
1,1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,2015-01-11,2015-01-11,1/11/2015,...,4,1.0,0,2.202658,19545.0,dr5rusxf6em6,dr5rgbzz1sgp,40.754078,-73.980087,0.741899
2,2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,2015-01-16,2015-01-16,1/16/2015,...,3,0.0,0,0.766369,19545.0,dr5ru1qt291p,dr5rspdbs3nh,40.754078,-73.980087,0.996261
3,3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,2015-01-16,2015-01-16,1/16/2015,...,3,0.0,0,1.057838,19545.0,dr5ru6363v2z,dr5rsre1umy7,40.754078,-73.980087,0.640517
4,4,-73.945511,40.773724,-73.987434,40.755707,1422173586,705,2015-01-25,2015-01-25,1/25/2015,...,8,1.0,0,2.522606,19545.0,dr5rvqds0tn9,dr5ru77gpy6w,40.754078,-73.980087,2.262122


In [164]:
Train_with_weather_Trip_date_hav_geo_dtw_dirc = create_downtown_direction_feature(Train_with_weather_Trip_date_hav_geo_dtw)

In [165]:
Train_with_weather_Trip_date_hav_geo_dtw_dirc.head()

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,date_id_tmp,date_id_left,Date,...,start_isWeekend,start_isHoliday,haversineDist,geoDist,start_geohash,end_geohash,downtown_lat,downtown_lng,DistanceToDowntown,DirectionToDowntown
0,0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,2015-01-11,2015-01-11,1/11/2015,...,1.0,0,0.493774,3803.0,dr5regsppmbb,dr5reuzen5x4,40.754078,-73.980087,3.169278,3.641055
1,1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,2015-01-11,2015-01-11,1/11/2015,...,1.0,0,2.202658,19545.0,dr5rusxf6em6,dr5rgbzz1sgp,40.754078,-73.980087,0.741899,0.679870
2,2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,2015-01-16,2015-01-16,1/16/2015,...,0.0,0,0.766369,19545.0,dr5ru1qt291p,dr5rspdbs3nh,40.754078,-73.980087,0.996261,4.038270
3,3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,2015-01-16,2015-01-16,1/16/2015,...,0.0,0,1.057838,19545.0,dr5ru6363v2z,dr5rsre1umy7,40.754078,-73.980087,0.640517,4.266397
4,4,-73.945511,40.773724,-73.987434,40.755707,1422173586,705,2015-01-25,2015-01-25,1/25/2015,...,1.0,0,2.522606,19545.0,dr5rvqds0tn9,dr5ru77gpy6w,40.754078,-73.980087,2.262122,0.927372


In [166]:
Train_with_weather_Trip_date_hav_geo_dtw_dirc.to_csv(r'D:\Learn\HomeWork_Lyft\train_ready.csv',index=False)

In [ ]:
Train_with_weather_Trip_date_hav_geo_dtw_dirc.columns()

## Exploratory Data Analysis

#### Categorical features

In [ ]:
duration_by_hour = Train_with_weather_Trip_date_hav_geo_dtw_dirc.groupby(['start_hour'], as_index=False)['duration'].mean()
duration_by_dayOfWeek = Train_with_weather_Trip_date_hav_geo_dtw_dirc.groupby(['start_dayOfWeek'], as_index=False)['duration'].mean()
duration_by_IsWeekend = Train_with_weather_Trip_date_hav_geo_dtw_dirc.groupby(['start_isWeekend'], as_index=False)['duration'].mean()
duration_by_IsHoliday = Train_with_weather_Trip_date_hav_geo_dtw_dirc.groupby(['start_isHoliday'], as_index=False)['duration'].mean()



#duration_by_dayOfWeek_hour = Train_with_weather_Trip_date_hav_geo_dtw_dirc.groupby(['start_dayOfWeek','start_hour'], as_index=False)['duration'].mean()

In [ ]:
plt.bar(duration_by_hour['start_hour'], duration_by_hour['duration'], align='center', alpha=0.5)
plt.xticks(duration_by_hour['start_hour'], np.arange(len(duration_by_hour['start_hour'])))
plt.ylabel('Average Duration')
plt.xlabel('Trip Start Hour')
plt.title('Average Duration Vs Hour')
plt.show()

In [ ]:
plt.bar(duration_by_dayOfWeek['start_dayOfWeek'], duration_by_dayOfWeek['duration'], align='center', alpha=0.5)
plt.xticks(duration_by_dayOfWeek['start_dayOfWeek'], np.arange(len(duration_by_dayOfWeek['start_dayOfWeek'])))
plt.ylabel('Average Duration')
plt.xlabel('Trip Start Day of Week: Monday = 0, Sunday = 6')
plt.title('Average Duration Vs Trip Start Day of Week')
plt.show()

In [ ]:
plt.bar(duration_by_IsWeekend['start_isWeekend'], duration_by_IsWeekend['duration'], align='center', alpha=0.5)
plt.xticks(duration_by_IsWeekend['start_isWeekend'], np.arange(len(duration_by_IsWeekend['start_isWeekend'])))
plt.ylabel('Average Duration')
plt.xlabel('Is Weekend?')
plt.title('Average Duration Vs Is Weekend?')
plt.show()

In [ ]:
plt.bar(duration_by_IsHoliday['start_isHoliday'], duration_by_IsHoliday['duration'], align='center', alpha=0.5)
plt.xticks(duration_by_IsHoliday['start_isHoliday'], np.arange(len(duration_by_IsHoliday['start_isHoliday'])))
plt.ylabel('Average Duration')
plt.xlabel('Is Holiday?')
plt.title('Average Duration Vs Is Holiday?')
plt.show()

#### Continuous features

In [ ]:
plt.scatter(Train_with_weather_Trip_date_hav_geo_dtw_dirc['Avg_Temp'], Train['duration'])
plt.xlabel('Avg_Temp')
plt.ylabel('duration')
plt.title('Average vs duration')
plt.show()

In [ ]:
plt.scatter(Train_with_weather_Trip_date_hav_geo_dtw_dirc['Snow_Depth'], Train['duration'])
plt.xlabel('Snow_Depth')
plt.ylabel('duration')
plt.title('Snow Depth vs duration')
plt.show()